<a href="https://colab.research.google.com/github/yoshimelaine/learn/blob/master/compound_data_acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [1]:
! pip install chembl_webresource_client

Install anaconda and RDkit in order to work on molecular data

In [2]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/') 

--2020-08-12 08:13:11--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.4’

Miniconda3-py37_4.8 100%[===================>]  81.12M   214MB/s    in 0.4s    

2020-08-12 08:13:12 (214 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.4’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to

**Importing libraries**

In [3]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import math
from rdkit.Chem import PandasTools

**Search for target protein**

In [4]:
target = new_client.target
molecule = new_client.molecule
bioactivities = new_client.activity

In [5]:
uniprot_id = 'P00533'
# Get target information from ChEMBL but restrict to specified values only
target_query = target.get(target_components__accession=uniprot_id) \
                     .only('target_chembl_id', 'organism', 'pref_name', 'target_type')
#print(type(target_query))
targets = pd.DataFrame.from_records(target_query)
targets

,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,Epidermal growth factor receptor erbB1,CHEMBL203,SINGLE PROTEIN
1,Homo sapiens,Epidermal growth factor receptor erbB1,CHEMBL203,SINGLE PROTEIN
2,Homo sapiens,Epidermal growth factor receptor and ErbB2 (HE...,CHEMBL2111431,PROTEIN FAMILY
3,Homo sapiens,Epidermal growth factor receptor,CHEMBL2363049,PROTEIN FAMILY
4,Homo sapiens,MER intracellular domain/EGFR extracellular do...,CHEMBL3137284,CHIMERIC PROTEIN



Select CHEMBL203 as the target of interest.

CHEMBL203: It is a single protein and represents the human Epidermal growth factor receptor (EGFR, also named erbB1)


In [6]:
selected_target = targets.target_chembl_id[0]

We want to download all molecules that have been tested against our target of interest,, only consider:
- human proteins
- bioactivity type IC50
- exact measurements (relation '=')
- binding data (assay type 'B')

In [7]:
bioact = bioactivities.filter(target_chembl_id = selected_target).filter(type = 'IC50').filter(relation = '=').filter(assay_type = 'B') \
                      .only('activity_id','assay_chembl_id','canonical_smiles', 'assay_description', 'assay_type', \
                            'molecule_chembl_id', 'type', 'units', 'relation', 'value', \
                            'target_chembl_id', 'target_organism')
len(bioact), len(bioact[0]), type(bioact), type(bioact[0])
bioact_df = pd.DataFrame.from_records(bioact)
bioact_df.head(10)

,activity_id,assay_chembl_id,assay_description,assay_type,canonical_smiles,molecule_chembl_id,relation,target_chembl_id,target_organism,type,units,value
0,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,uM,0.041
1,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,uM,0.041
2,32267,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,CHEMBL69960,=,CHEMBL203,Homo sapiens,IC50,uM,0.17
3,32680,CHEMBL677833,In vitro inhibition of Epidermal growth factor...,B,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,CHEMBL137635,=,CHEMBL203,Homo sapiens,IC50,uM,9.3
4,32770,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,CHEMBL306988,=,CHEMBL203,Homo sapiens,IC50,uM,500.0
5,32772,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,O=C(O)/C=C/c1ccc(O)cc1,CHEMBL66879,=,CHEMBL203,Homo sapiens,IC50,uM,3000.0
6,32780,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],CHEMBL77085,=,CHEMBL203,Homo sapiens,IC50,uM,96.0
7,33406,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,CHEMBL443268,=,CHEMBL203,Homo sapiens,IC50,uM,5.31
8,34039,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,CHEMBL76979,=,CHEMBL203,Homo sapiens,IC50,uM,264.0
9,34041,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,N#CC(C#N)=C(N)/C(C#N)=C/c1ccc(O)cc1,CHEMBL76589,=,CHEMBL203,Homo sapiens,IC50,uM,0.125


In [8]:
bioact_df.shape

(7178, 12)

In [9]:
bioact_df = bioact_df.dropna(axis=0, how='any') #delete entries with missing values

In [10]:
bioact_df = bioact_df.drop_duplicates('molecule_chembl_id', keep='first') #Delete duplicates: Sometimes the same molecule (molecule_chembl_id) has been tested more than once

In [11]:
bioact_df.shape

(5498, 12)

In [12]:
print(bioact_df.units.unique())
bioact_df = bioact_df.drop(bioact_df.index[~bioact_df.units.str.contains('M')])
print(bioact_df.units.unique())
bioact_df.shape

['uM' 'nM' 'M' "10'1 ug/ml" 'ug ml-1' "10'-1microM" "10'1 uM"
 "10'-1 ug/ml" "10'-2 ug/ml" "10'2 uM" '/uM' "10'-6g/ml" 'mM' 'umol/L'
 'nmol/L']
['uM' 'nM' 'M' "10'-1microM" "10'1 uM" "10'2 uM" '/uM' 'mM']


(5428, 12)

In [13]:
bioact_df = bioact_df.reset_index(drop=True) 
bioact_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,canonical_smiles,molecule_chembl_id,relation,target_chembl_id,target_organism,type,units,value
0,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,uM,0.041
1,32267,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,CHEMBL69960,=,CHEMBL203,Homo sapiens,IC50,uM,0.17
2,32680,CHEMBL677833,In vitro inhibition of Epidermal growth factor...,B,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,CHEMBL137635,=,CHEMBL203,Homo sapiens,IC50,uM,9.3
3,32770,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,CHEMBL306988,=,CHEMBL203,Homo sapiens,IC50,uM,500.0
4,32772,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,O=C(O)/C=C/c1ccc(O)cc1,CHEMBL66879,=,CHEMBL203,Homo sapiens,IC50,uM,3000.0


In [12]:
bioact_df.to_csv('bioactivity-raw.csv', index = False)

In [13]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [14]:
! cp bioactivity-raw.csv "/content/gdrive/My Drive/Colab Notebooks/Data"

Convert all value units to nM

In [14]:
def convert_to_NM(unit, bioactivity):
  value = bioact_df['value']
  if unit != "nM":
        if unit == "pM":
            value = float(bioactivity)/1000
        elif unit == "10'-11M":
            value = float(bioactivity)/100
        elif unit == "10'-10M":
            value = float(bioactivity)/10
        elif unit == "10'-8M":
            value = float(bioactivity)*10
        elif unit == "10'-1microM" or unit == "10'-7M":
            value = float(bioactivity)*100
        elif unit == "uM" or unit == "/uM" or unit == "10'-6M":
            value = float(bioactivity)*1000
        elif unit == "10'1 uM":
            value = float(bioactivity)*10000
        elif unit == "10'2 uM":
            value = float(bioactivity)*100000
        elif unit == "mM":
            value = float(bioactivity)*1000000
        elif unit == "M":
            value = float(bioactivity)*1000000000
        else:
            print ('unit not recognized...', unit)
        return value
  else: return bioactivity

In [15]:
bioactivity_nM = []
for i, row in bioact_df.iterrows():
    bioact_nM = convert_to_NM(row['units'], row['value'])
    bioactivity_nM.append(bioact_nM)
bioact_df['value'] = bioactivity_nM
bioact_df['units'] = 'nM'
bioact_df.head(10)

,activity_id,assay_chembl_id,assay_description,assay_type,canonical_smiles,molecule_chembl_id,relation,target_chembl_id,target_organism,type,units,value
0,32260,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,=,CHEMBL203,Homo sapiens,IC50,nM,41
1,32267,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,CHEMBL69960,=,CHEMBL203,Homo sapiens,IC50,nM,170
2,32680,CHEMBL677833,In vitro inhibition of Epidermal growth factor...,B,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,CHEMBL137635,=,CHEMBL203,Homo sapiens,IC50,nM,9300
3,32770,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,CHEMBL306988,=,CHEMBL203,Homo sapiens,IC50,nM,500000
4,32772,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,O=C(O)/C=C/c1ccc(O)cc1,CHEMBL66879,=,CHEMBL203,Homo sapiens,IC50,nM,3e+06
5,32780,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],CHEMBL77085,=,CHEMBL203,Homo sapiens,IC50,nM,96000
6,33406,CHEMBL674637,Inhibitory activity towards tyrosine phosphory...,B,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,CHEMBL443268,=,CHEMBL203,Homo sapiens,IC50,nM,5310
7,34039,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,CHEMBL76979,=,CHEMBL203,Homo sapiens,IC50,nM,264000
8,34041,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,N#CC(C#N)=C(N)/C(C#N)=C/c1ccc(O)cc1,CHEMBL76589,=,CHEMBL203,Homo sapiens,IC50,nM,125
9,34049,CHEMBL674643,Inhibitory concentration of EGF dependent auto...,B,N#CC(C#N)=Cc1ccc(O)c(O)c1,CHEMBL76904,=,CHEMBL203,Homo sapiens,IC50,nM,35000


In [16]:
bioact_df.shape

(5428, 12)

In [17]:
selection = ['molecule_chembl_id','canonical_smiles','type','value','units']
df1 = bioact_df[selection]
df1.head(10)

,molecule_chembl_id,canonical_smiles,type,value,units
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,IC50,41,nM
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,IC50,170,nM
2,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,IC50,9300,nM
3,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,IC50,500000,nM
4,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,IC50,3e+06,nM
5,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],IC50,96000,nM
6,CHEMBL443268,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,IC50,5310,nM
7,CHEMBL76979,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,IC50,264000,nM
8,CHEMBL76589,N#CC(C#N)=C(N)/C(C#N)=C/c1ccc(O)cc1,IC50,125,nM
9,CHEMBL76904,N#CC(C#N)=Cc1ccc(O)c(O)c1,IC50,35000,nM


Convert the IC50 values to pIC50

In [18]:
df2 = df1.rename(columns= {'value':'value_IC50'})
df2.head(10)

,molecule_chembl_id,canonical_smiles,type,value_IC50,units
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,IC50,41,nM
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,IC50,170,nM
2,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,IC50,9300,nM
3,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,IC50,500000,nM
4,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,IC50,3e+06,nM
5,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],IC50,96000,nM
6,CHEMBL443268,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,IC50,5310,nM
7,CHEMBL76979,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,IC50,264000,nM
8,CHEMBL76589,N#CC(C#N)=C(N)/C(C#N)=C/c1ccc(O)cc1,IC50,125,nM
9,CHEMBL76904,N#CC(C#N)=Cc1ccc(O)c(O)c1,IC50,35000,nM


In [19]:
df2 = df2[~df2['canonical_smiles'].isnull()]
print(df2.shape)
df2.head()

(5428, 5)


,molecule_chembl_id,canonical_smiles,type,value_IC50,units
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,IC50,41,nM
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,IC50,170,nM
2,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,IC50,9300,nM
3,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,IC50,500000,nM
4,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,IC50,3e+06,nM


To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).

This custom function pIC50() will accept a DataFrame as input and will:

- Take the IC50 values from the value_IC50 column and converts it from nM to M by multiplying the value by 10$^{-9}$
- Take the molar value and apply -log10
- Delete the value_IC50 column and create a new value_pIC50 column


In [25]:
df3 = df2.reset_index(drop=True)

In [26]:
# dfx = df2.head(5)
ic50 = df3['value_IC50'].astype(float)

In [27]:
print(len(ic50))
print(ic50.head(10))

5428
0         41.0
1        170.0
2       9300.0
3     500000.0
4    3000000.0
5      96000.0
6       5310.0
7     264000.0
8        125.0
9      35000.0
Name: value_IC50, dtype: float64


In [28]:
pIC50 = pd.Series() 
i = 0
while i < len(df3.value_IC50):
    value = 9 - math.log10(ic50[i]) # pIC50=-log10(IC50 mol/l) --> for nM: -log10(IC50*10**-9)= 9-log10(IC50)
    if value < 0:
        print("Negative pIC50 value at index"+str(i))
    pIC50.at[i] = value
    i += 1
    
df3['value_pIC50'] = pIC50
df3.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,molecule_chembl_id,canonical_smiles,type,value_IC50,units,value_pIC50
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,IC50,41,nM,7.387216
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,IC50,170,nM,6.769551
2,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,IC50,9300,nM,5.031517
3,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,IC50,500000,nM,3.301030
4,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,IC50,3e+06,nM,2.522879


In [29]:
selection = ['molecule_chembl_id','canonical_smiles','value_IC50','value_pIC50','units']
df = df3[selection]
df.head(10)

,molecule_chembl_id,canonical_smiles,value_IC50,value_pIC50,units
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,41,7.387216,nM
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,170,6.769551,nM
2,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,9300,5.031517,nM
3,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,500000,3.301030,nM
4,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,3e+06,2.522879,nM
5,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],96000,4.017729,nM
6,CHEMBL443268,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,5310,5.274905,nM
7,CHEMBL76979,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,264000,3.578396,nM
8,CHEMBL76589,N#CC(C#N)=C(N)/C(C#N)=C/c1ccc(O)cc1,125,6.903090,nM
9,CHEMBL76904,N#CC(C#N)=Cc1ccc(O)c(O)c1,35000,4.455932,nM


In [32]:
df.to_csv('filtered-data.csv', index = False)

In [33]:
! cp filtered-data.csv "/content/gdrive/My Drive/Colab Notebooks/Data"